#### Accessing Pyomo variable values and objective function value

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.core import Var

In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

#### Abstract model

In [3]:
# Transport problem
model = AbstractModel()
# Define the sets
model.markets = Set()
model.factories = Set()
# Define the variables
model.production = Param(model.factories)
model.demand = Param(model.markets)
# Define the matrix of cost of send one profuct from factories to markets 
model.costs = Param(model.factories,model.markets)
# Define the variable of units of product from factory $i$ to market $j$ $x_{i,j}$
model.units = Var(model.factories, model.markets, within = NonNegativeReals)
# Define objective function (1)
def Totalcost(model):
    return sum(model.costs[n,i]*model.units[n,i]
               for n in model.factories
               for i in model.markets)
# Define objective 
model.finalcost = Objective(rule = Totalcost)
# Define constraint function 
def MinDemand(model,markets):
    return sum(model.units[i,markets] for i in model.factories) >= model.demand[markets]
# Define constraint function 
def MaxProduction(model,factories):
    return sum(model.units[factories,j] for j in model.markets) <= model.production[factories]
# Define constraints 
model.DemandConstraint = Constraint(model.markets, rule = MinDemand)
model.ProductionConstraint = Constraint(model.factories, rule = MaxProduction)

In [4]:
opt = SolverFactory('glpk')

In [5]:
instance = model.create_instance('data.dat') 

In [6]:
results = opt.solve(instance, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmpizlvmt45.glpk.raw
 --wglp /var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmpi_790k8_.glpk.glp
 --cpxlp /var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmps9hbrlsu.pyomo.lp
Reading problem data from '/var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmps9hbrlsu.pyomo.lp'...
10 rows, 21 columns, 41 non-zeros
119 lines were read
Writing problem data to '/var/folders/9k/04pt01hs1ng95lppnvj97vw40000gn/T/tmpi_790k8_.glpk.glp'...
105 lines were written
GLPK Simplex Optimizer, v4.65
10 rows, 21 columns, 41 non-zeros
Preprocessing...
9 rows, 20 columns, 40 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 9
      0: obj =   0.000000000e+00 inf =   7.250e+02 (5)
      6: obj =   4.281750000e+03 inf =   0.000e+00 (0)
*    11: o

In [7]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 2328.75
  Upper bound: 2328.75
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 21
  Number of nonzeros: 41
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04532670974731445
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [8]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2328.75
  Upper bound: 2328.75
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 21
  Number of nonzeros: 41
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04532670974731445
# ----------------------------------------------------------
#   Solution Information
# ------------------------------

In [9]:
instance.solutions.load_from(results)
for v in instance.component_objects(Var, active=True):
    print ("Variable",v)
    varobject = getattr(instance, str(v))
    for index in varobject:
        print ("   ",index, varobject[index].value)

Variable units
    ('Madrid', 'Alicante') 0.0
    ('Madrid', 'Castellon') 0.0
    ('Madrid', 'Vitoria') 75.0
    ('Madrid', 'Cadiz') 0.0
    ('Madrid', 'Caceres') 175.0
    ('Barcelona', 'Alicante') 0.0
    ('Barcelona', 'Castellon') 150.0
    ('Barcelona', 'Vitoria') 25.0
    ('Barcelona', 'Cadiz') 0.0
    ('Barcelona', 'Caceres') 0.0
    ('Almeria', 'Alicante') 150.0
    ('Almeria', 'Castellon') 0.0
    ('Almeria', 'Vitoria') 0.0
    ('Almeria', 'Cadiz') 150.0
    ('Almeria', 'Caceres') 0.0
    ('Coruna', 'Alicante') 0.0
    ('Coruna', 'Castellon') 0.0
    ('Coruna', 'Vitoria') 0.0
    ('Coruna', 'Cadiz') 0.0
    ('Coruna', 'Caceres') 0.0
